In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [2]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from calendar import c
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time
from math import sqrt

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles() -> dict:

        return {
            "CybP_01": False,  # | Компрометация кода АП | - неверный хэш автопилота
            "CybP_02": False,  # | Неполадки связи | - смешное сообщение
            "CybP_03": False,  # | Привода, поддел инфы о них | - если взлом, то 'eeeeeeee'
            "CybP_04": False,  # | Компроментация навиг. системы | - если взлом, то выдает неверны данные
            "CybP_05": False,  # | Компроментация навесного оборудования | - высокая темп-ра или скорость щетки
            "CybP_06": False,  # | Компроментация МТС | - меняется передаваемый пакет и маршрут движения
        }

    def get_photo(self):  # Для отладки компьютерного зрения [debug] #
        from datetime import datetime
        import os
        import cv2
        import numpy as np

        frame = super().get_camera_frame()
        
        if frame is not None:
            save_dir = "../../Photos"

            # Генерируем уникальное имя файла с временной меткой
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # миллисекунды
            
            # Сохраняем изображение
            image_filename = os.path.join(save_dir, f"frame_{timestamp}.jpg")
            cv2.imwrite(image_filename, frame)
            self.lg.log(f"Сохранение изображения")
            
            # Текстовый файл (только цвета пикселей)
            array_filename = os.path.join(save_dir, f"frame_{timestamp}.txt")
            np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
            self.lg.log(f"Сохранение массива")
        
        return frame


        
        return True  # пока что всегда возвращает True

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 20
        green = False
        red = False
        self.lg.log("🚦 Внимание! Светофор 🚦")

        while True:
            frame = self.get_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(40-velocity, 40+velocity)) and (g in range(200-velocity, 200+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(40-velocity, 40+velocity)) and (g in range(40-velocity, 40+velocity)) and (r in range(200-velocity, 200+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                if not green:
                    green = True
                    self.lg.log("🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢")
                break
            else:
                if not red:
                    red = True
                    self.lg.log("🔴 Светофор КРАСНЫЙ. Ожидаем... 🔴")
                time.sleep(1.0)

        return True 

    def check_pedestrian_CV(self) -> bool:  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (188, 193, 196)  # новые значения bgr
        x_1, y_1 = 11, 309
        x_2, y_2 = 507, 352
        x_3, y_3 = 490, 390
        x_4, y_4 = 3, 317
        velocity = 20
        wait = False
        
        while True:
            frame = self.get_camera_frame()
            pixels_1 = frame[y_1, x_1-10:x_1+10]
            pixels_2 = frame[y_2, x_2-5:x_2+5]
            pixels_3 = frame[y_3, x_3-5:x_3+5]
            pixels_4 = frame[y_4, x_4-5:x_4+5]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_1]
            mas_2 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_2]
            mas_3 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_3]
            mas_4 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels_4]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            result_3 = any([all(i) for i in mas_3])
            result_4 = any([all(i) for i in mas_4])
            
            if result_1 or result_2 or result_3:
                self.lg.log(f"🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️")
                break
            else:
                if not wait:
                    wait = True
                    self.lg.log(f"🚸 Пешеходный переход. Ожидаем... 🚸")
                continue
        return True

    def change_barrier_CV(self) -> int:  # Смена положения шлагбаума [КЗ] #
        
        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30

        def check_status():
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    return 0  # закрыт
            return 1  # открыт
        
        start_status = check_status()
        repeated_failure = False
        if not start_status:
            while True:
                self.set_barrier_toggle()
                time.sleep(2.0)
                if check_status():
                    self.lg.log("✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅")
                    break
                else:
                    if not repeated_failure:
                        self.lg.log("❌ Устройство переключения шлагбаума работает нестабильно ❌")
                        repeated_failure = True
                    else:
                        self.lg.log("   ❌🎲 ПОВТОРНОЕ не срабатывание устройства переключения 🎲❌")
        else:
            self.lg.log("✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅")

    def check_autobot_UDP(self) -> bool:  # Проверка положения автобота перехватом трафика [UDP] #
        buffer = self.get_message_from_trusted_module()
        while True:
            buffer += self.get_message_from_trusted_module()
            time.sleep(0.5)
            if "start" in buffer:
                break

    def mission_code(self):
        
        def robot_marshrut():
            self.set_robot_speed(0.24)
            self.set_brush_speed(100)
            # self.change_barrier_UDP("drop")  # Сразу закрываем шлагбаум дистанционно (уже так не сделать)
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.68, "y": 2.25})  # Пешеход верхний
            self.check_pedestrian_CV()
            # self.change_barrier_UDP("drop")  # Снова закрываем шлагбаум дистанционно (уже так не сделать)

            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # Возможное столкновение с роботом (первый чек)
            self.check_autobot_UDP()  # Проверяет удаляется и приближается автобот
            
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.7, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.16, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
            self.ap_hook.do_move({"x": 1.6, "y":3.08}) #
            self.ap_hook.do_rotate({"x": 1.32, "y":2.12}) #
            self.ap_hook.do_move({"x": 1.32, "y":2.12}) #

            self.ap_hook.do_rotate({"x": 1.32, "y":2.12})
            self.ap_hook.do_move({"x": 1.32, "y":2.12})
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 2.6, "y":2.12})
            self.ap_hook.do_rotate({"x": 2.65, "y":2.2})
            self.ap_hook.do_move({"x": 2.65, "y":2.2})
            self.ap_hook.do_rotate({"x": 2.6, "y":2.28})
            self.ap_hook.do_move({"x": 2.6, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.2, "y":2.28})
            self.ap_hook.do_move({"x": 1.8, "y":2.28})  # Светофор 1 (центральный)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.36, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.32, "y":0.8})
            self.ap_hook.do_move({"x": 1.32, "y":1})  # Светофор 0 (нижний)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.32, "y":0.8})
            self.ap_hook.do_rotate({"x": 1.4, "y":0.6})
            self.ap_hook.do_move({"x": 1.4, "y":0.6})
            self.ap_hook.do_rotate({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1.4})
            self.ap_hook.do_move({"x": 1.48, "y":1.8})
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.6})  # Возможное столкновение с роботом (второй чек)
            self.check_autobot_UDP()  # Проверяет удаляется и приближается автобот

            self.ap_hook.do_move({"x": 1.48, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.ap_hook.do_move({"x": 0.52, "y":2.95})  # Верхний пешеход
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.52, "y":1.8})
            self.ap_hook.do_move({"x": 0.52, "y":1.4})
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.ap_hook.do_rotate({"x": 0.52, "y":0.52})
            self.ap_hook.do_move({"x": 0.52, "y":0.52})
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1, "y":0.52})

            self.ap_hook.do_move({"x": 1.85, "y":0.52})  # Пешеход нижний 
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.ap_hook.do_move({"x": 3.08, "y":0.52})
            self.ap_hook.do_rotate({"x": 3.08, "y":1})  # Возможное столкновение с роботом (третий чек)
            self.check_autobot_UDP()  # Проверяет удаляется и приближается автобот

            self.ap_hook.do_move({"x": 3.08, "y":1})
            self.ap_hook.do_move({"x": 3.08, "y":1.4})
            self.ap_hook.do_move({"x": 3.08, "y":1.8})
            self.ap_hook.do_move({"x": 3.08, "y":2.2})
            self.ap_hook.do_move({"x": 3.08, "y":2.6})
            self.ap_hook.do_move({"x": 3.08, "y":3.0})
            # self.check_autobot_UDP() !!! НЕ ЗДЕСЬ !!!
            self.ap_hook.do_rotate({"x": 3, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.92, "y":3})
            self.ap_hook.do_move({"x": 2.92, "y":3})
            self.ap_hook.do_rotate({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":0.68})
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
            self.ap_hook.do_move({"x": 0.6, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0}) 

        robot_marshrut()



class UserTrustedHandler(TrustedHandler):
    
    def __init__(self, context):
        super().__init__(context)

    class UdpMaster:
        
        mode = None

        @classmethod
        def check_mode(cls) -> str:
                
            def th_chmode():
                port = 10004
                listen_port = port
                interceptor_socket = None
                
                try:
                    interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                    interceptor_socket.settimeout(2.0)
                    interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                    interceptor_socket.bind(('0.0.0.0', listen_port))
                    
                    try:
                        data = interceptor_socket.recvfrom(1024)[0]
                        decoded_data = data.decode('utf-8', errors='ignore').strip()
                        result = decoded_data

                        interceptor_socket.close()
                        cls.mode = "twin"
                        return

                    except socket.timeout:
                        cls.mode = "real"
                        
                    except Exception as e:
                        pass
                        
                except Exception as e:
                    pass
            
            interceptor_th = threading.Thread(target=th_chmode, daemon=True)
            interceptor_th.start()

        @classmethod
        def coords_autobot_UDP(cls, trusted_obj):

            ports = {
            "twin": 10004,  # в двойнике
            "real": 8000   # реальное железо
            }

            listen_port = ports[f"{cls.mode}"]
            result = None
            dangerous = False
            flag = False
            
            def connect():
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                interceptor_socket.bind(('0.0.0.0', listen_port))
                
                data = interceptor_socket.recvfrom(1024)[0]
                decoded_data = data.decode('utf-8', errors='ignore').strip()
                result = decoded_data
                interceptor_socket.close()
                return result

            while True:
                try:
                    check = []
                    my_x, my_y = trusted_obj.real_position()
                    for _ in range(2):
                        if _ == 1:
                            time.sleep(1.5)
                        result = connect()
                        data_autorobot = result[1:-1].split(',')
                        coords = []
                        for i in range(2):
                            coord = float(data_autorobot[i].split(':')[1])
                            coords.append(coord)
                        autorobot_x, autorobot_y = coords[0], coords[1]
                        d = round(sqrt((autorobot_x - my_x)**2 + (autorobot_y - my_y)**2), 4)
                        check.append(d)
                    
                    cell_number = trusted_obj._calculate_cell_from_position(autorobot_x, autorobot_y)
                    if cell_number in (6, 7, 8, 15, 17):
                        continue

                    if check[0] == check[1]:
                        if dangerous:  # ехал и остановился
                            for _ in range(5):  # чтобы наверняка
                                trusted_obj.send_message_to_ap("start")
                            trusted_obj.lg.log("[📡<OP>🛜]::💤 Авторобот остановился, можно ехать ♻️")
                            break
                        if not flag:  # стоит на месте
                            trusted_obj.lg.log("[📡<OP>🛜]::💤 Пока что авторобот стоит на месте. Ожидаем... 🛑")
                            flag = True
                    elif check[1] > check[0]:  # расстояние увеличивается
                        for _ in range(5):  # чтобы наверняка
                            trusted_obj.send_message_to_ap("start")
                        trusted_obj.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении - он уезжает от нас! Вперед... 🟢")
                        break
                    else:  # расстояние сокращается
                        if not dangerous:
                            trusted_obj.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении. Ожидаем... 🛑")
                            dangerous = True
                                    
                except Exception as e:
                    trusted_obj.lg.log(f"Ошибка создания сокета: {e}")

            time.sleep(5.0)  # чтобы АП успел прочитать буфер
        
        @classmethod
        def intercept_UDP(cls, trusted_obj: UserMissionHandler, name_device: str) -> int:  # Перехват трафика умных устройств [UDP] #

            ip_addreses_twin = "127.0.0.1"
            ports_twin = {
                "traffic_0": 5041,  # светофор 0 (самый ближний)
                "traffic_1": 5042,  # светофр 1 (средний)
                "traffic_2": 5043,  # светофор 2 (самый дальний)
                "barrier": 5031,   # шлагбаум (больше не работает)
                "autobot": 5071    # МТС - компроментация 
            }

            ip_addreses_real = {
                "traffic_0": "192.168.60.181",  # светофор 0 (самый ближний)
                "traffic_1": "192.168.60.182",  # светофр 1 (средний)
                "traffic_2": "192.168.60.183",  # светофор 2 (самый дальний)
                "barrier": "192.168.60.131",   # шлагбаум (больше не работает)
                "autobot": "192.168.60.145"    # МТС - компроментация 
            }
            port_real = 4000

            if cls.mode == "twin":
                ip_adress = ip_addreses_twin
                listen_port = ports_twin[name_device]
            else:
                ip_adress = ip_addreses_real["name_device"]
                listen_port = port_real

            result = None 
            interceptor_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                interceptor_socket.bind((ip_adress, listen_port))
                
                try:
                    data = interceptor_socket.recvfrom(1024)[0]
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name_device in tuple(ports_twin.keys())[:-2]:  # светофоры
                        color = int(decoded_data[-2])
                        trusted_obj.lg.log(f"{color}")
                        trusted_obj.lg.log(f"{name_device} КРАСНЫЙ") if color == 1 else trusted_obj.lg.log(f"{name_device} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name_device in tuple(ports_twin.keys())[-2]:  # шлагбаум
                        status = decoded_data[-2]
                        result = int(status)
                    elif name_device in tuple(ports_twin.keys())[-1]:  # автоматичский робот
                        result = decoded_data

                    interceptor_socket.close()
                    return result
                    
                except Exception as e:
                    trusted_obj.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                trusted_obj.lg.log(f"Ошибка создания сокета: {e}")


    def trusted_code(self):
        
        self._check_robot_position()
        self.UdpMaster.check_mode()
        time.sleep(4.0)
        self.lg.log(f'{self.UdpMaster.mode}')

    def real_position(self):  # вычиселние реальной поизиции

        data = self.get_robot_status()
        x_a, y_a = data["base_station_alpha_rssi"]
        x_b, y_b = data["base_station_bravo_rssi"]
        x_c, y_c = data["base_station_charlie_rssi"]
        
        r1_sq = x_a**2 + y_a**2
        r2_sq = x_b**2 + y_b**2
        r3_sq = x_c**2 + y_c**2
        
        C1 = r1_sq - r2_sq + 2
        C2 = r1_sq - r3_sq - 2.43
        
        x = -(C2 + 0.1 * C1) / 1.4
        y = x + C1 / 2
        
        return x/1000, y/1000

    def make_next_short_message(self, prev_message: str):
        pass
        self.get_robot_status()
        some_message = "exactly_correct_short_message"
        return some_message

    def check_CybP_06_UDP(self) -> bool:  # Проверка компроментации МТС (CybP_06) [UDP] #
        
        data = self.UdpMaster.intercept_UDP(self, "autobot")
        result = int(data[-2])
        repeated_failure = False
        if result:
            if not repeated_failure:
                repeated_failure = True
                self.lg.log('[📡<OP>🛜]::🛑 Обнаружена компроментация МТС! 🛑')
        else:
            repeated_failure = False
            self.lg.log('[📡<OP>🛜]::🍀 Компроментации МТС не обнаружено. 🍀')
        
        return True  # пока что всегда возвращает True  

### 

    def _check_robot_position(self):
            
        def monitor():
            while True:
                robot_status_for_rotation = self.get_robot_status()
                current_x, current_y = self.real_position()
                rotation = robot_status_for_rotation["rotation"]
                self._calculate_cell_from_position(current_x, current_y, int(rotation))
                time.sleep(1.5)

        monitor_start = threading.Thread(target=monitor, daemon=True)
        monitor_start.start()

    def _calculate_cell_from_position(self, x, y, rotation=500):
        cell_x = x // 0.4 + 1
        cell_y = 9 - (y // 0.4 + 1)
        
        cell_number = int(cell_y * 9 + cell_x)

        # МТС
        if (cell_number == 12 and -10 <= rotation <= 10) \
            or (cell_number == 22 and 80 <= rotation <= 100) \
                or (cell_number == 71 and 80 <= rotation <= 100):
            if not self.current_cell:
                self.current_cell = True
                self.check_CybP_06_UDP()
                self.UdpMaster.coords_autobot_UDP(self)
        elif cell_number not in (12, 22, 71):
            self.current_cell = False

        return cell_number


In [ ]:
start_module(UserMissionHandler, UserTrustedHandler)

[2025-10-18 22:12:38]::INIT::AGTS-WRAP-CLIENT, версия 2.1.0
[2025-10-18 22:12:38]::INIT::UDP(c) <- (uid=dawP) инициализирован.
[2025-10-18 22:12:38]::INIT::UDP(c) -> (uid=yIqn) инициализирован.
[2025-10-18 22:12:38]::INIT::UDP(c) <- (uid=guRf) инициализирован.
[2025-10-18 22:12:38]::INIT::Загрузка завершена.
[2025-10-18 22:12:38]::INIT::UDP(c) -> (uid=alAh) инициализирован.
[2025-10-18 22:12:38]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[2025-10-18 22:12:38]::INFO::(AP) Заезд инициализирован - ожидание старта
[2025-10-18 22:12:38]::INFO::(TM) Заезд инициализирован - ожидание старта
[2025-10-18 22:12:41]::INFO::Доверенный код инициализирован
[2025-10-18 22:12:41]::INFO::Код заезда инициализирован
[2025-10-18 22:12:45]::INFO::twin
[2025-10-18 22:12:52]::INFO::✅ Шлагбаум ОТКРЫТ. Можно продолжать движение. ✅
[2025-10-18 22:12:57]::INFO::🚸 Пешеходный переход. Ожидаем... 🚸
[2025-10-18 22:13:43]::INFO::🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️
[2025-10-18 22:13:55]::INFO::

[2025-10-18 22:19:59]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:00]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:01]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:02]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:03]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:04]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:05]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:06]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:07]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:08]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:09]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-10-18 22:20:10]::ERROR::Ошибка отправки команды: АСО не отвечает
